In [2]:
!pip install datasets gdown
!pip install torch einops
!pip install 'numpy<2'
!pip install -U sentence-transformers
!pip install 'accelerate>=0.26.0'
!pip install gdown
!pip install -q pyvi|

!git clone https://github.com/vncorenlp/VnCoreNLP.git
!pip install py_vncorenlp
import py_vncorenlp
py_vncorenlp.download_model(save_dir='/kaggle/working/VnCoreNLP')
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/kaggle/working/VnCoreNLP')
%cd /kaggle/working/
# !gdown --id 1STFX5W2vwJVMWeYJ24To95f0sX8Erf_Y --output train_data_org.csv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.3.1
    Uninstalling sentence-transformers-3.3.1:
      Successfully uninstalled sentence-transformers-3.3.1
/bin/bash: -c: line 2: syntax error: unexpected end of file
Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 259 (delta 17), reused 33 (delta 11), pack-reused 212 (from 1)
Receiving objects: 100% (259/259), 237.79 MiB | 36.95 MiB/s, done.
Resolving deltas: 100% (93/93), done.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 34.4 MB/s eta 0:00:0000:01
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4305 sha256=965a879d152ed4e8353e2b0501b252bd9a0321547e42014fce664d4ebe5defd1
  Stored in directo

In [3]:
import pandas as pd
import ast

In [4]:
corpus = pd.read_csv("/kaggle/input/fix-tokenize-corpus/corpus.csv")
train_org = pd.read_csv("/kaggle/input/split-data/train.csv")
#train_org = pd.read_csv("/kaggle/input/split-data/test.csv")

In [5]:
def process_context(context):
    return [
        item.strip('[]\'\"').replace('\\n', '').strip('"')
        for item in context.split('\n')
        if item.strip()
    ]

def process_cid(cid_str):
    return [int(x) for x in cid_str.strip('[]').split()]

# Áp dụng các hàm vào DataFrame
train_org['cid'] = train_org['cid'].apply(process_cid)
train_org['context'] = train_org['context'].apply(process_context)

In [6]:
for idx, row in train_org.iterrows():
    for i, (cid, _) in enumerate(zip(row['cid'], row['context'])):
        if cid in corpus['cid'].values: 
            train_org.at[idx, 'context'][i] = corpus.loc[corpus['cid'] == cid, 'text'].values[0]

for idx, row in train_org.iterrows():
    for i, (cid, context) in enumerate(zip(row['cid'], row['context'])):
        if cid not in corpus['cid'].values: 
            train_org.at[idx, 'context'][i] = " ".join(rdrsegmenter.word_segment(context))

In [7]:
with_mine = pd.read_csv("/kaggle/input/mine-negative-sen-bkai/train_with_negatives_4.csv")
columns_to_convert = ["negative_cid", "negative_context"]

for col in columns_to_convert:
    with_mine[col] = with_mine[col].apply(ast.literal_eval)
with_mine.head()

,question,context,cid,qid,negative_cid,negative_context
0,Người học ngành quản lý khai thác công trình t...,"['Khả năng học tập, nâng cao trình độ\\n- Khối...",[62492],161615,"[35066, 134143, 65675, 242472, 188807, 147732,...",[Nâng bậc lương1. Người quản lý công ty chuyên...
1,Nội dung lồng ghép vấn đề bình đẳng giới trong...,['Nội dung lồng ghép vấn đề bình đẳng giới tro...,[151154],80037,"[461115, 214112, 591097, 26114, 151153, 53897,...",[Khoản 3.1- Nâng cao nhận thức về công tác phụ...
2,Sản phẩm phần mềm có được hưởng ưu đãi về thời...,"['Điều 20. Ưu đãi về thời gian miễn thuế, giảm...",[75071],124074,"[557258, 157135, 17873, 73603, 506982, 631699,...",[Điều 7. Tiêu chí cụ thể đối với sản phẩm phần...
3,Điều kiện để giáo viên trong cơ sở giáo dục mầ...,"['Điều kiện được hưởng\\nCán bộ quản lý, giáo ...",[225897],146841,"[70683, 14311, 521812, 442161, 552188, 613651,...","[I. PHẠM VI, ĐỐI TƯỢNG VÀ ĐIỀU KIỆN ÁP DỤNG1. ..."
4,Nguyên tắc áp dụng phụ cấp ưu đãi nghề y tế th...,['Nguyên tắc áp dụng\\n1. Trường hợp công chức...,[68365],6176,"[172835, 449105, 34366, 207806, 126900, 126677...",[Về thực hiện chính sách tiền lương...Từ ngày ...


In [7]:
print(len(with_mine))

119456


In [8]:
final = train_org.merge(
    with_mine[['qid', 'negative_cid', 'negative_context']],
    on='qid',
    how='left'
)

In [9]:
final['question'] = final['question'].apply(lambda context: " ".join(rdrsegmenter.word_segment(context)))
final.head()

,question,context,cid,qid,negative_cid,negative_context
0,Người học ngành quản_lý khai_thác công_trình t...,"[Khả_năng học_tập , nâng cao trình_độ - Khối_l...",[62492],161615,"[35066, 134143, 65675, 242472, 188807, 147732,...",[Nâng bậc lương1. Người quản lý công ty chuyên...
1,Nội_dung lồng_ghép vấn_đề bình_đẳng giới trong...,[Nội_dung lồng_ghép vấn_đề bình_đẳng giới tron...,[151154],80037,"[461115, 214112, 591097, 26114, 151153, 53897,...",[Khoản 3.1- Nâng cao nhận thức về công tác phụ...
2,Sản_phẩm phần_mềm có được hưởng ưu_đãi về thời...,"["" Điều 20 . Ưu_đãi về thời_gian miễn thuế , g...",[75071],124074,"[557258, 157135, 17873, 73603, 506982, 631699,...",[Điều 7. Tiêu chí cụ thể đối với sản phẩm phần...
3,Điều_kiện để giáo_viên trong cơ_sở giáo_dục mầ...,"[Điều_kiện được hưởng Cán_bộ quản_lý , giáo_vi...",[225897],146841,"[70683, 14311, 521812, 442161, 552188, 613651,...","[I. PHẠM VI, ĐỐI TƯỢNG VÀ ĐIỀU KIỆN ÁP DỤNG1. ..."
4,Nguyên_tắc áp_dụng phụ_cấp ưu_đãi nghề y_tế th...,"[Nguyên_tắc áp_dụng 1 . Trường_hợp công_chức ,...",[68365],6176,"[172835, 449105, 34366, 207806, 126900, 126677...",[Về thực hiện chính sách tiền lương...Từ ngày ...


In [10]:
# import pandas as pd
# import re

# # Hàm áp dụng regex trên từng chuỗi trong danh sách
# def apply_regex_to_list(lst, pattern, repl):
#     return [re.sub(pattern, repl, item) for item in lst]

# # Áp dụng cho từng cột
# final['negative_context'] = final['negative_context'].apply(
#     lambda x: apply_regex_to_list(x, r'(\D)(\d)', r'\1 \2')
# )
# final['context'] = final['context'].apply(
#     lambda x: apply_regex_to_list(x, r'(\d+)(\D)', r'\1 \2')
# )
# final.head()

,question,context,cid,qid,negative_cid,negative_context
0,Phó Tổng_Giám_đốc Ngân_hàng Chính_sách_xã_hội ...,[Áp_dụng chế_độ tiền_lương và phụ_cấp quy_định...,[140864],70867,"[528765, 572673, 514251, 583125, 17301, 618705...","[Điều 8. Mức tiền lương bình quân kế hoạch, q..."
1,Ai có thẩm_quyền quyết_định thành_lập Hội_đồng...,[Thành_lập Hội_đồng 1 . Bộ_trưởng Bộ Y_tế ra ...,[62339],813,"[550254, 217782, 485071, 74532, 626731, 30585,...","[Điều 12. Hội đồng tư vấn, Hội đồng quản lý 1..."
2,Thời_hiệu xử_phạt đối_với nhà_xuất_bản thực_hi...,[Điều 5 . Thời_hiệu xử_phạt vi_phạm hành_chín...,[63171],40392,"[169898, 67492, 566524, 217447, 551061, 472409...",[Thời hiệu xử lý vi phạm hành chính 1. Thời hi...
3,"Việc ký_kết , thực_hiện thoả_thuận quốc_tế nhâ...",[Báo_cáo tình_hình ký_kết và thực_hiện thoả_th...,[157761],85946,"[218667, 168253, 511073, 510475, 565009, 63166...","[Trách nhiệm của đơn vị tổ chức hội nghị, hội ..."
4,Đề_án sử_dụng tài_sản công tại đơn_vị sự_nghiệ...,"["" Điều 44 . Đề_án sử_dụng tài_sản công tại đ...",[95397],55607,"[62810, 448562, 74995, 505955, 505959, 76871, ...",[Sử dụng tài sản công tại đơn vị sự nghiệp côn...


In [11]:
# final.head()

,question,context,cid,qid,negative_cid,negative_context
0,Phó Tổng_Giám_đốc Ngân_hàng Chính_sách_xã_hội ...,[Áp_dụng chế_độ tiền_lương và phụ_cấp quy_định...,[140864],70867,"[528765, 572673, 514251, 583125, 17301, 618705...","[Điều 8. Mức tiền lương bình quân kế hoạch, q..."
1,Ai có thẩm_quyền quyết_định thành_lập Hội_đồng...,[Thành_lập Hội_đồng 1 . Bộ_trưởng Bộ Y_tế ra ...,[62339],813,"[550254, 217782, 485071, 74532, 626731, 30585,...","[Điều 12. Hội đồng tư vấn, Hội đồng quản lý 1..."
2,Thời_hiệu xử_phạt đối_với nhà_xuất_bản thực_hi...,[Điều 5 . Thời_hiệu xử_phạt vi_phạm hành_chín...,[63171],40392,"[169898, 67492, 566524, 217447, 551061, 472409...",[Thời hiệu xử lý vi phạm hành chính 1. Thời hi...
3,"Việc ký_kết , thực_hiện thoả_thuận quốc_tế nhâ...",[Báo_cáo tình_hình ký_kết và thực_hiện thoả_th...,[157761],85946,"[218667, 168253, 511073, 510475, 565009, 63166...","[Trách nhiệm của đơn vị tổ chức hội nghị, hội ..."
4,Đề_án sử_dụng tài_sản công tại đơn_vị sự_nghiệ...,"["" Điều 44 . Đề_án sử_dụng tài_sản công tại đ...",[95397],55607,"[62810, 448562, 74995, 505955, 505959, 76871, ...",[Sử dụng tài sản công tại đơn vị sự nghiệp côn...


In [10]:
from tqdm import tqdm

# First loop with tqdm
for idx, row in tqdm(final.iterrows(), total=final.shape[0], desc="Updating negative context (found in corpus)"):
    for i, (cid, _) in enumerate(zip(row['negative_cid'], row['negative_context'])):
        if cid in corpus['cid'].values: 
            final.at[idx, 'negative_context'][i] = corpus.loc[corpus['cid'] == cid, 'text'].values[0]

# Second loop with tqdm
for idx, row in tqdm(final.iterrows(), total=final.shape[0], desc="Updating negative context (not found in corpus)"):
    for i, (cid, context) in enumerate(zip(row['negative_cid'], row['negative_context'])):
        if cid not in corpus['cid'].values: 
            final.at[idx, 'negative_context'][i] = " ".join(rdrsegmenter.word_segment(context))


Updating negative context (not found in corpus): 100%|██████████| 109456/109456 [02:45<00:00, 661.66it/s]


In [13]:
def process_context(context):
    return [
        item.strip('[]\'\"').replace('\\n', '').strip('"')
        for item in context
        if item.strip()
    ]

final['negative_context'] = final['negative_context'].apply(process_context)

In [14]:
final.head()

,question,context,cid,qid,negative_cid,negative_context
0,Người học ngành quản_lý khai_thác công_trình t...,"[Khả_năng học_tập , nâng cao trình_độ - Khối_l...",[62492],161615,"[35066, 134143, 65675, 242472, 188807, 147732,...",[Nâng bậc lương 1 . Người_quản_lý công_ty chuy...
1,Nội_dung lồng_ghép vấn_đề bình_đẳng giới trong...,[Nội_dung lồng_ghép vấn_đề bình_đẳng giới tron...,[151154],80037,"[461115, 214112, 591097, 26114, 151153, 53897,...",[Khoản 3.1 - Nâng cao nhận_thức về công_tác ph...
2,Sản_phẩm phần_mềm có được hưởng ưu_đãi về thời...,"["" Điều 20 . Ưu_đãi về thời_gian miễn thuế , g...",[75071],124074,"[557258, 157135, 17873, 73603, 506982, 631699,...",[Điều 7 . Tiêu_chí cụ_thể đối_với sản_phẩm phầ...
3,Điều_kiện để giáo_viên trong cơ_sở giáo_dục mầ...,"[Điều_kiện được hưởng Cán_bộ quản_lý , giáo_vi...",[225897],146841,"[70683, 14311, 521812, 442161, 552188, 613651,...","[ I . PHẠM_VI , ĐỐI_TƯỢNG VÀ ĐIỀU_KIỆN ÁP_DỤNG..."
4,Nguyên_tắc áp_dụng phụ_cấp ưu_đãi nghề y_tế th...,"[Nguyên_tắc áp_dụng 1 . Trường_hợp công_chức ,...",[68365],6176,"[172835, 449105, 34366, 207806, 126900, 126677...",[Về thực_hiện chính_sách tiền_lương ... Từ ngà...


In [27]:
final.iloc[7]

question            Hương_ước có nội_dung trái với phong_tục tập_q...
context             [" Điều 15 . Tạm ngừng thực_hiện hương_ước , q...
cid                                                    [31682, 31677]
qid                                                             60201
negative_cid        [139430, 539402, 111513, 31686, 539413, 124161...
negative_context    [ Điều 9 . Công_nhận hương_ước , quy_ước [ ......
Name: 7, dtype: object

In [ ]:
final.iloc[3]['negative_cid']

In [32]:
final.iloc[7]['context'][0]

' Điều 15 . Tạm ngừng thực_hiện hương_ước , quy_ước 1 . Hương_ước , quy_ước bị tạm ngừng thực_hiện một phần hoặc toàn_bộ khi thuộc một trong các trường_hợp sau đây : a ) Có nội_dung quy_định tại điểm a Khoản 1 Điều 10 của Quyết_định này và nếu áp_dụng sẽ gây ra thiệt_hại hoặc có nguy_cơ gây hậu_quả nghiêm_trọng ; b ) Chưa được Uỷ_ban_nhân_dân cấp huyện công_nhận mà vẫn thực_hiện . 2 . Khi có căn_cứ quy_định tại điểm a hoặc điểm b Khoản 1 Điều này , trong thời_hạn 03 ngày làm_việc , Uỷ_ban_nhân_dân cấp huyện ra quyết_định tạm ngừng thực_hiện hương_ước , quy_ước ; chỉ_đạo Uỷ_ban_nhân_dân cấp xã hướng_dẫn thôn , tổ_dân_phố tiến_hành việc sửa_đổi , bổ_sung , thay_thế khi thuộc trường_hợp quy_định tại điểm a Khoản 1 Điều này hoặc tiến_hành các thủ_tục đề_nghị công_nhận theo quy_định khi thuộc trường_hợp quy_định tại điểm b Khoản 1 Điều này . 3 . Quyết_định tạm ngừng thực_hiện hương_ước , quy_ước phải quy_định rõ thời_hạn thực_hiện việc sửa_đổi , bổ_sung , thay_thế hoặc hoàn_tất các thủ_tục 

In [21]:
len(final)

10000

In [33]:
final.to_csv('test_cross_rand_3.csv', index=False, encoding='utf-8')
